In [ ]:
import mlflow
import mlflow.pytorch

import torch
import torch.nn as nn
import numpy as np

In [ ]:
mlflow.set_experiment('MLP Experiment')
def mlflow_run_decorator(run_name=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            mlflow.start_run(run_name=run_name)
            try:
                result = func(*args, **kwargs)
                mlflow.set_tag("Status", "SUCCEESS")
            except Exception as e:
                mlflow.log_param("Exception", e)
                mlflow.set_tag("Status", "FAIL")
                raise e
            finally:
                mlflow.end_run()
            return result
        return wrapper
    return decorator

In [ ]:
X = np.random.rand(100, 1)
y = 1 + 2 * X + .1 * np.random.randn(100, 1)

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [ ]:
class Config:
    EPOCHS = 100
    LEARNING_RATE = 0.1

CONFIG = Config()

In [ ]:
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=CONFIG.LEARNING_RATE)

In [ ]:
@mlflow_run_decorator(run_name='Error Test Run')
def train_model():
    mlflow.log_params(vars(CONFIG))
    
    for epoch in range(CONFIG.EPOCHS):
        inputs = torch.from_numpy(X.astype(np.float32))
        labels = torch.from_numpy(y.astype(np.float32))

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        mlflow.log_metric('loss', loss.item(), step=epoch)
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}/{CONFIG.EPOCHS}, Loss: {loss.item()}')

In [ ]:
train_model()